# Ejercicio 1: Generador de Caminatas Aleatorias (Componente de DeepWalk)

El algoritmo DeepWalk aprende representaciones vectoriales (embeddings) de nodos en un grafo basándose en secuencias de nodos generadas por caminatas aleatorias. Una **caminata aleatoria** comienza en un nodo y se mueve a un vecino elegido al azar, repitiendo este proceso durante un número determinado de pasos.

Tu tarea es implementar una función en Python llamada `generar_caminata_aleatoria`.

1.  **`generar_caminata_aleatoria(grafo: nx.Graph, nodo_inicio: int, longitud_camino: int) -> list`**:
    *   Esta función toma tres argumentos:
        *   `grafo`: Un objeto grafo de NetworkX (`nx.Graph`).
        *   `nodo_inicio`: El nodo desde el cual comenzará la caminata.
        *   `longitud_camino`: La longitud máxima de la caminata (número de nodos en la secuencia).
    *   La función debe:
        1.  Inicializar la caminata con el `nodo_inicio`.
        2.  Repetir `longitud_camino - 1` veces:
            a.  Obtener la lista de vecinos del nodo actual en la caminata.
            b.  **Importante**: De estos vecinos, filtrar aquellos que ya han sido visitados *en la caminata actual* para evitar ciclos inmediatos y fomentar la exploración.
            c.  Si no quedan vecinos válidos (no visitados en la caminata actual), la caminata se detiene prematuramente.
            d.  Elegir aleatoriamente uno de los vecinos válidos.
            e.  Añadir el vecino elegido a la caminata y actualizar el nodo actual al vecino elegido.
    *   Debe devolver una lista de nodos que representa la caminata aleatoria generada. Por ejemplo, `[nodo_inicio, nodo2, nodo3, ...]`.

In [1]:
import networkx as nx
import random
from node2vec import Node2Vec
from gensim.models import Word2Vec, KeyedVectors
from sklearn.decomposition import PCA

def generar_caminata_aleatoria_solucion(grafo: nx.Graph, nodo_inicio: int, longitud_camino: int) -> list:
    # your code here
    caminata = [nodo_inicio]
    nodo_actual = nodo_inicio

    # Repetimos hasta alcanzar la longitud deseada
    for _ in range(longitud_camino - 1):
        # Vecinos del nodo actual
        vecinos = list(grafo.neighbors(nodo_actual))
        # Filtrar vecinos que ya aparecieron en la caminata (evitar revisitar)
        vecinos_validos = [v for v in vecinos if v not in caminata]

        # Si no hay vecinos válidos, se detiene la caminata
        if not vecinos_validos:
            break

        # Elegir aleatoriamente uno de los vecinos válidos
        siguiente_nodo = random.choice(vecinos_validos)

        # Agregar a la caminata y actualizar nodo actual
        caminata.append(siguiente_nodo)
        nodo_actual = siguiente_nodo

    return caminata

g_test_caminata = nx.path_graph(5)
nodo_inicial_test = 0
longitud_test = 4
random.seed(10)
caminata_generada = generar_caminata_aleatoria_solucion(g_test_caminata, nodo_inicial_test, longitud_test)
assert isinstance(caminata_generada, list) 
assert len(caminata_generada) > 0 
assert caminata_generada[0] == nodo_inicial_test
assert len(caminata_generada) <= longitud_test

# Ejercicio 2: Generación de Corpus de Caminatas (Componente de DeepWalk)

Para que DeepWalk aprenda representaciones efectivas, necesita un "corpus" de muchas caminatas aleatorias. Este corpus se genera iniciando múltiples caminatas desde cada nodo (o un subconjunto de ellos) en el grafo.

Tu tarea es implementar una función en Python llamada `generar_corpus_caminatas`. Esta función utilizará la función `generar_caminata_aleatoria` que implementaste en el ejercicio anterior (o una versión de solución si la tuya no está disponible).

1.  **`generar_corpus_caminatas(grafo: nx.Graph, numero_caminatas_por_nodo: int, longitud_camino: int) -> list`**:
    *   Esta función toma tres argumentos:
        *   `grafo`: Un objeto grafo de NetworkX (`nx.Graph`).
        *   `numero_caminatas_por_nodo`: El número de caminatas aleatorias que se deben generar comenzando desde *cada* nodo en el grafo.
        *   `longitud_camino`: La longitud deseada para cada caminata individual (que se pasará a `generar_caminata_aleatoria`).
    *   La función debe:
        1.  Iterar sobre cada nodo en el `grafo`.
        2.  Para cada nodo, generar `numero_caminatas_por_nodo` caminatas aleatorias utilizando la función `generar_caminata_aleatoria` (que implementaste previamente). Cada una de estas caminatas debe comenzar en el nodo actual de la iteración y tener la `longitud_camino` especificada.
        3.  Recopilar todas las caminatas generadas en una sola lista.
    *   Debe devolver una lista de listas, donde cada lista interna es una caminata (una secuencia de nodos). Por ejemplo: `[[camino1_nodoA], [camino2_nodoA], [camino1_nodoB], ...]`.

In [2]:
def generar_corpus_caminatas_solucion(grafo: nx.Graph, numero_caminatas_por_nodo: int, longitud_camino: int) -> list:
    # your code here
    corpus = []

    # Iterar sobre cada nodo del grafo
    for nodo in grafo.nodes():
        # Generar múltiples caminatas desde este nodo
        for _ in range(numero_caminatas_por_nodo):
            caminata = generar_caminata_aleatoria_solucion(grafo, nodo, longitud_camino)
            corpus.append(caminata)

    return corpus
    
g_test_corpus = nx.Graph([(0,1), (1,2)]) # Nodos 0, 1, 2
num_caminatas_test = 2
longitud_camino_test = 3
random.seed(20)
corpus_generado = generar_corpus_caminatas_solucion(g_test_corpus, num_caminatas_test, longitud_camino_test)
expected_total_walks = g_test_corpus.number_of_nodes() * num_caminatas_test
assert isinstance(corpus_generado, list)
assert len(corpus_generado) == expected_total_walks
assert len(corpus_generado[0]) > 0
assert corpus_generado[0][0] in g_test_corpus.nodes()
assert len(corpus_generado[0]) <= longitud_camino_test

# Ejercicio 3: Generando Embeddings de Nodos con la Librería Node2Vec

**Node2Vec** es un algoritmo que aprende representaciones vectoriales (embeddings) de baja dimensionalidad para nodos en un grafo. Extiende la idea de DeepWalk controlando la estrategia de caminata aleatoria mediante dos parámetros, `p` y `q`, que permiten balancear entre exploración tipo BFS (homofilia) y DFS (características estructurales).

Tu tarea es implementar una función en Python llamada `obtener_embeddings_node2vec`.

1.  **`obtener_embeddings_node2vec(grafo: nx.Graph, dimensiones: int, longitud_caminata: int, num_caminatas: int, p_param: float = 1, q_param: float = 1, workers_param: int = 1) -> Word2Vec.wv`**:
    *   Esta función toma un grafo y varios parámetros para Node2Vec.
    *   Debe:
        1.  Inicializar un modelo `Node2Vec` (de la librería `node2vec`) con el `grafo` y los parámetros proporcionados (`dimensions`, `walk_length`, `num_walks`, `p`, `q`, `workers`). Asegúrate de que el parámetro `quiet` esté en `True` para evitar salidas excesivas.
        2.  Entrenar el modelo Node2Vec usando el método `fit()`. No necesitas especificar `window`, `min_count` o `sg` aquí, ya que `fit` los pasará al modelo Word2Vec subyacente con valores por defecto o los que la librería `node2vec` determine.
        3.  Devolver los embeddings aprendidos. Estos se pueden acceder a través de `model.wv` (donde `model` es la instancia de Node2Vec después de entrenar). El tipo de retorno esperado es `gensim.models.keyedvectors.KeyedVectors`.

Observación: La librería `node2vec` internamente utiliza `gensim.models.Word2Vec` para aprender los embeddings a partir de las caminatas que genera.

In [3]:
def obtener_embeddings_node2vec_solucion(grafo: nx.Graph, 
                                         dimensiones: int, 
                                         longitud_caminata: int, 
                                         num_caminatas: int, 
                                         p_param: float = 1, 
                                         q_param: float = 1, 
                                         workers_param: int = 1) -> KeyedVectors:
    # your code here
    # Inicializar el modelo Node2Vec
    node2vec = Node2Vec(
        grafo,
        dimensions=dimensiones,
        walk_length=longitud_caminata,
        num_walks=num_caminatas,
        p=p_param,
        q=q_param,
        workers=workers_param,
        quiet=True  # para evitar logs ruidosos
    )
    
    # Entrenar el modelo (usa los parámetros por defecto de Word2Vec)
    model = node2vec.fit()
    
    # Devolver los embeddings (KeyedVectors)
    return model.wv
    
g_test_n2v = nx.karate_club_graph() 
dims_test = 16
len_walk_test = 10
num_walks_test = 5 
random.seed(42) # Asegurar reproducibilidad para Node2Vec también
embeddings_wv = obtener_embeddings_node2vec_solucion(g_test_n2v, 
                                                 dimensiones=dims_test, 
                                                 longitud_caminata=len_walk_test, 
                                                 num_caminatas=num_walks_test)

assert embeddings_wv is not None, "La función no devolvió embeddings (model.wv)."
assert len(embeddings_wv.vectors) == g_test_n2v.number_of_nodes(), \
    f"El número de vectores ({len(embeddings_wv.vectors)}) no coincide con el número de nodos ({g_test_n2v.number_of_nodes()})."
assert embeddings_wv.vector_size == dims_test, \
    f"La dimensionalidad de los vectores ({embeddings_wv.vector_size}) no coincide con la esperada ({dims_test})."